In [ ]:
import cv2
import numpy as np
import time

def create_background(cap,num_frames=30):
    print('move out of the frame')
    background=[]
    for i in range(num_frames):
        ret,frame=cap.read()
        if ret:
            background.append(frame)
        else:
            print('could not capture frame')
        time.sleep(0.1)
    if background:
        np.median(background,axis=0).astype(np.uint8)
    else:
        raise ValueError('could not capture any frame')

def create_mask(frame,lower_color,upper_color):
    #HSV frame is easier to filter colors
    hsvimg=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)#frame is converted from RGB to HSV
    mask=cv2.inRange(hsvimg,lower_color,upper_color)#if hsv frame is within the color range we've specified then marked as white else as black
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,np.ones((3,3),np.uint8),iterations=2)
    return mask

def cloak_effect(frame,mask,background):
    mask_inv=cv2.bitwise_not(mask)
    fg=cv2.bitwise_and(frame,frame,mask=mask_inv)
    bg=cv2.bitwise_and(background,background,mask=mask)
    return cv2.add(fg,bg)

def main():
    cap=cv2.VideoCapture(0)
    if not cap.isOpened():
        print('error:camera couldnt open')
        return
    try:
        background=create_background(cap)
    except ValueError as e:
        print(f'error:{e}')
        cap.release()
        return
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])
    print('enter q to quit. camera starting to read')
    while True:
        ret,frame=cap.read()
        if not ret:
            print('couldnt read frame')
            time.sleep(1)
            continue
        mask=create_mask(frame,lower_blue,upper_blue)
        result=cloak_effect(frame,mask,background)
        cv2.imshow('invisible cloak',result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
if __name__ == "__main__":
    main()





